# **Handwritten digit recognition using MNIST and Convolutional Neural Networks (CNNs)**

In these exercises, we will use PyTorch to implement Deep Learning methods that perform pattern recognition. In particular, we will consider a Convolutional Neural Network (CNN) to recognize MNIST's handwritten digits in this article.

We will see three ways of using CNNs to classify MNIST data:


1.   Training a custom model for feature extraction and classification.
2.   Pretrained CNN as a feature extractor, combined with a shallow classifier.
3.   Fine tuning CNN to perform both feature extraction and classification.


In [1]:
from torchvision import models
from torchvision import transforms
from torchvision import datasets
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import time
import copy


ModuleNotFoundError: No module named 'torchvision'

In [ ]:
batch_sizeP = 300
numWorkersP = 2